In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests

#API Key. The file is located in the same folder as the notebook but has not been pushed to GitHub
from config import ninja_api

import json
import psycopg2
import matplotlib.pyplot as plt

In [2]:
#Url for API Ninja.Country
api_url = 'https://api.api-ninjas.com/v1/country?name='

In [3]:
#Reading in the CSV with the vaccination data
owid_df = pd.read_csv('./Resources/owid-covid-data.csv')
owid_df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [4]:
#Filtering out the columns of the data set that are not related to vaccines
red_owid_df = owid_df.copy()
red_owid_df = red_owid_df[['iso_code','continent', 'location', 'date', 'population', 
                           'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters']] 
red_owid_df.rename(columns={'location':'country'}, inplace=True)
red_owid_df

,iso_code,continent,country,date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters
0,AFG,Asia,Afghanistan,2020-02-24,39835428.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,39835428.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,39835428.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,39835428.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,39835428.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
163782,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,4336161.0,3367921.0,106474.0
163783,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,4338919.0,3371563.0,110403.0
163784,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,4340412.0,3372616.0,110866.0
163785,ZWE,Africa,Zimbabwe,2022-02-21,15092171.0,7829096.0,4342121.0,3374371.0,112604.0


In [5]:
clean_df = red_owid_df.copy()

#Dropping all rows which do not have vaccination data
clean_df.dropna(inplace=True)
clean_df.reset_index(drop = True, inplace = True)
clean_df

,iso_code,continent,country,date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters
0,ALB,Europe,Albania,2021-11-03,2872934.0,1926604.0,1025180.0,898625.0,2799.0
1,ALB,Europe,Albania,2021-11-06,2872934.0,1944676.0,1031588.0,904858.0,8230.0
2,ALB,Europe,Albania,2021-11-07,2872934.0,1947148.0,1032457.0,905876.0,8815.0
3,ALB,Europe,Albania,2021-11-09,2872934.0,1963359.0,1037546.0,910845.0,14968.0
4,ALB,Europe,Albania,2021-11-10,2872934.0,1971973.0,1040015.0,913524.0,18434.0
...,...,...,...,...,...,...,...,...,...
12019,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,4336161.0,3367921.0,106474.0
12020,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,4338919.0,3371563.0,110403.0
12021,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,4340412.0,3372616.0,110866.0
12022,ZWE,Africa,Zimbabwe,2022-02-21,15092171.0,7829096.0,4342121.0,3374371.0,112604.0


In [7]:
clean_country_df = clean_df.copy()

#Iterating through every row to clean up the country names, taking out any with parts in paranthesis.
#The iso_code is unique to every nation so any conflicts over this will be resolved through that
for n, row in clean_country_df.iterrows():
    row['country'] = row['country'].split('(', 1)[0]
clean_country_df

,iso_code,continent,country,date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters
0,ALB,Europe,Albania,2021-11-03,2872934.0,1926604.0,1025180.0,898625.0,2799.0
1,ALB,Europe,Albania,2021-11-06,2872934.0,1944676.0,1031588.0,904858.0,8230.0
2,ALB,Europe,Albania,2021-11-07,2872934.0,1947148.0,1032457.0,905876.0,8815.0
3,ALB,Europe,Albania,2021-11-09,2872934.0,1963359.0,1037546.0,910845.0,14968.0
4,ALB,Europe,Albania,2021-11-10,2872934.0,1971973.0,1040015.0,913524.0,18434.0
...,...,...,...,...,...,...,...,...,...
12019,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,4336161.0,3367921.0,106474.0
12020,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,4338919.0,3371563.0,110403.0
12021,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,4340412.0,3372616.0,110866.0
12022,ZWE,Africa,Zimbabwe,2022-02-21,15092171.0,7829096.0,4342121.0,3374371.0,112604.0


In [8]:
perc_df = clean_country_df.copy()
#Creating new columns where the percentage of the population with vaccines is calculated
perc_df['perc_vaccinated'] = perc_df['people_vaccinated']/perc_df['population']
perc_df['perc_fully_vaccinated'] = perc_df['people_fully_vaccinated']/perc_df['population']
perc_df

,iso_code,continent,country,date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,perc_vaccinated,perc_fully_vaccinated
0,ALB,Europe,Albania,2021-11-03,2872934.0,1926604.0,1025180.0,898625.0,2799.0,0.356841,0.312790
1,ALB,Europe,Albania,2021-11-06,2872934.0,1944676.0,1031588.0,904858.0,8230.0,0.359071,0.314960
2,ALB,Europe,Albania,2021-11-07,2872934.0,1947148.0,1032457.0,905876.0,8815.0,0.359374,0.315314
3,ALB,Europe,Albania,2021-11-09,2872934.0,1963359.0,1037546.0,910845.0,14968.0,0.361145,0.317043
4,ALB,Europe,Albania,2021-11-10,2872934.0,1971973.0,1040015.0,913524.0,18434.0,0.362004,0.317976
...,...,...,...,...,...,...,...,...,...,...,...
12019,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,4336161.0,3367921.0,106474.0,0.287312,0.223157
12020,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,4338919.0,3371563.0,110403.0,0.287495,0.223398
12021,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,4340412.0,3372616.0,110866.0,0.287594,0.223468
12022,ZWE,Africa,Zimbabwe,2022-02-21,15092171.0,7829096.0,4342121.0,3374371.0,112604.0,0.287707,0.223584


In [9]:
final_df = perc_df.copy()
#Dropping the columns with the total number of people vaccinated/fully vaccinated due to redundeny
final_df.drop(['people_vaccinated', 'people_fully_vaccinated'], axis=1, inplace=True)
final_df.index.names = ['id']
final_df

,iso_code,continent,country,date,population,total_vaccinations,total_boosters,perc_vaccinated,perc_fully_vaccinated
id,,,,,,,,,
0,ALB,Europe,Albania,2021-11-03,2872934.0,1926604.0,2799.0,0.356841,0.312790
1,ALB,Europe,Albania,2021-11-06,2872934.0,1944676.0,8230.0,0.359071,0.314960
2,ALB,Europe,Albania,2021-11-07,2872934.0,1947148.0,8815.0,0.359374,0.315314
3,ALB,Europe,Albania,2021-11-09,2872934.0,1963359.0,14968.0,0.361145,0.317043
4,ALB,Europe,Albania,2021-11-10,2872934.0,1971973.0,18434.0,0.362004,0.317976
...,...,...,...,...,...,...,...,...,...
12019,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,106474.0,0.287312,0.223157
12020,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,110403.0,0.287495,0.223398
12021,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,110866.0,0.287594,0.223468


In [10]:
#Finding the list of unique iso_codes for use in the API
countries = final_df['iso_code'].unique()
countries

array(['ALB', 'DZA', 'AND', 'AIA', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE',
       'BHS', 'BHR', 'BGD', 'BLR', 'BEL', 'BLZ', 'BMU', 'BTN', 'BOL',
       'BRA', 'KHM', 'CAN', 'CHL', 'CHN', 'COL', 'CRI', 'CUB', 'CUW',
       'CYP', 'CZE', 'DNK', 'DOM', 'ECU', 'SLV', 'EST', 'FRO', 'FIN',
       'FRA', 'DEU', 'GIB', 'GRC', 'GRD', 'GTM', 'GGY', 'GUY', 'HND',
       'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRL', 'IMN', 'ISR',
       'ITA', 'JAM', 'JPN', 'JEY', 'KAZ', 'KEN', 'OWID_KOS', 'KWT', 'LVA',
       'LBN', 'LBY', 'LIE', 'LTU', 'LUX', 'MAC', 'MYS', 'MDV', 'MLT',
       'MCO', 'MNG', 'MNE', 'NPL', 'NLD', 'NZL', 'MKD', 'NOR', 'OMN',
       'PAK', 'PAN', 'PRY', 'PER', 'POL', 'PRT', 'RUS', 'KNA', 'VCT',
       'SMR', 'SAU', 'SRB', 'SGP', 'SVK', 'SVN', 'ZAF', 'KOR', 'ESP',
       'LKA', 'SUR', 'SWE', 'CHE', 'TWN', 'THA', 'TTO', 'TUN', 'TUR',
       'UKR', 'ARE', 'GBR', 'USA', 'URY', 'VEN', 'VNM', 'ZWE'],
      dtype=object)

In [11]:
#Creating an empty Dataframe to receive the country statistics retrieved through the API
countries_df = pd.DataFrame(columns = ['iso_code', 'GDP', 'Population Density', 'Urbanization', 'Unemployment'])

#Creating an empty list to store all iso codes that were found on the API
api_countries = []

#Iterating through the list of unique iso codes
for nat in countries:
    
    #Querying the API. ninja_api is the api key which is taken from a file called config.py.
    response = requests.get(api_url+nat, headers={'X-Api-Key':ninja_api}).json()
    
    #Checking if the call was successful
    try:
        #Creating a temporary dataframe to store the data from the API
        new_country_df = pd.DataFrame({'iso_code':[nat], 'GDP': [response[0]['gdp']], 
                                       'Population Density':[response[0]['pop_density']], 
                                       'Urbanization': [response[0]['urban_population']], 
                                       'Unemployment': [response[0]['unemployment']]})
        
        #Concatenating the temporary Dataframe onto the one meant to store the statistics
        countries_df = pd.concat([countries_df, new_country_df], ignore_index = True, axis = 0)
        
        #Adding the iso_code to the list of countries that we have statistics for
        api_countries.append(nat)
        
    except (IndexError, KeyError):
        
        #Printing a message if the API did not send a response for a specific iso_code and listing it
        print(f'{nat} is not a valid country for the API')

#Setting the index of the new dataframe to the iso code
countries_df.set_index('iso_code', inplace=True)
countries_df

AND is not a valid country for the API
AIA is not a valid country for the API
FRO is not a valid country for the API
GIB is not a valid country for the API
GRD is not a valid country for the API
GGY is not a valid country for the API
IMN is not a valid country for the API
JEY is not a valid country for the API
OWID_KOS is not a valid country for the API
MCO is not a valid country for the API
KNA is not a valid country for the API
TWN is not a valid country for the API


,GDP,Population Density,Urbanization,Unemployment
iso_code,,,,
ALB,15059.0,105.0,61.2,12.8
DZA,173757.0,18.4,73.2,11.5
ARG,518475.0,16.5,92.0,10.4
ARM,12433.0,104.1,63.2,16.6
AUS,1453871.0,3.3,86.1,5.3
...,...,...,...,...
USA,20580223.0,36.2,82.5,3.9
URY,59597.0,19.8,95.4,8.8
VEN,208338.0,32.2,88.2,9.4


In [12]:
#Creating a new dataframe which only includes that vaccination data for countries that were recognized by the API
purged_final_df = final_df.loc[final_df['iso_code'].isin(api_countries)]
purged_final_df

,iso_code,continent,country,date,population,total_vaccinations,total_boosters,perc_vaccinated,perc_fully_vaccinated
id,,,,,,,,,
0,ALB,Europe,Albania,2021-11-03,2872934.0,1926604.0,2799.0,0.356841,0.312790
1,ALB,Europe,Albania,2021-11-06,2872934.0,1944676.0,8230.0,0.359071,0.314960
2,ALB,Europe,Albania,2021-11-07,2872934.0,1947148.0,8815.0,0.359374,0.315314
3,ALB,Europe,Albania,2021-11-09,2872934.0,1963359.0,14968.0,0.361145,0.317043
4,ALB,Europe,Albania,2021-11-10,2872934.0,1971973.0,18434.0,0.362004,0.317976
...,...,...,...,...,...,...,...,...,...
12019,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,106474.0,0.287312,0.223157
12020,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,110403.0,0.287495,0.223398
12021,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,110866.0,0.287594,0.223468


In [13]:
#Establishing the connection to Postgresql
conn_string = 'postgres:postgres@localhost:5432/vaccinations_db'
engine = create_engine(f'postgresql://{conn_string}')

In [14]:
engine.table_names()

C:\Users\wawil\AppData\Local\Temp/ipykernel_21556/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['vaccinations', 'stats']

In [15]:
#Uploading the dataframes to the database
purged_final_df.to_sql(name='vaccinations', con = engine, if_exists='replace', index = True)

In [16]:
countries_df.to_sql(name='stats', con = engine, if_exists = 'replace', index = True)